# Notebook for: Comparing cities

# Objectives

Find the best place to locate a gaming company.

**Scheme of the company:**

- 20 Designers
- 5 UI/UX Engineers
- 10 Frontend Developers
- 15 Data Engineers
- 5 Backend Developers
- 20 Account Managers
- 1 Maintenance guy that loves basketball
- 10 Executives
- 1 CEO/President.


**Requirements:**  

- Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design.
- 30% of the company staff have at least 1 child.
- Developers like to be near successful tech startups that have raised at least 1 Million dollars.
- Executives like Starbucks A LOT. Ensure there's a starbucks not too far.
- Account managers need to travel a lot.
- Everyone in the company is between 25 and 40, give them some place to go party.
- The CEO is vegan.
- If you want to make the maintenance guy happy, a basketball stadium must be around 10 Km.
- The office dog—"Dobby" needs a hairdresser every month. Ensure there's one not too far away.

# Libraries

In [49]:
import os
import pandas as pd
import time
import re
import io
import requests
import json
from dotenv import load_dotenv
import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

# Foursquare API requests

I will be requesting information to **Foursquare** using the coordinates of the company that located at the center of the respective gaming-design-tech hub:
- San Francisco: Globant **(37.781929, -122.404176)** / Tagged (37.775300, -122.418600) / **Meez (37.785271, -122.397582)**
- New York: Cellufun (40.739930, -73.993049)
- London (GBR): spigit (51.517038, 0.139476)

**Loading token**

In [3]:
load_dotenv()

True

In [6]:
token_fsq = os.getenv("token_foursquare")

**Function to request from Foursquare**

In [94]:
def getFoursquare(query, location, radius, limit=1):
    '''This function requests information from Foursquare Place Search:
    :query: str, your query in Foursquare.
    :location: list, in the format: [latitude, longitude], both int.
    :radius: int, to limit the radius of the request.
    :limit: int, to limit the number of requests. By default 1.
    '''
        
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&radius={radius}&sort=DISTANCE&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_fsq,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

**Function to request from Foursquare specifying the categories**

In [120]:
def getFoursquareCategory(query, location, radius, categories, limit=1):
    '''This function requests information from Foursquare Place Search:
    :query: str, your query in Foursquare.
    :location: list, in the format: [latitude, longitude], both int.
    :radius: int, to limit the radius of the request.
    :categories: str, the category or categories ID, separated by commas.
    :limit: int, to limit the number of requests. By default 1.
    '''
        
    ll = f"{location[0]}%2C{location[1]}"
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={ll}&radius={radius}&categories={categories}&sort=DISTANCE&limit={str(limit)}"

    headers = {
        "accept": "application/json",
        "Authorization": token_fsq,
    }
    
    response = requests.get(url, headers=headers).json()
    
    return response

**Function to extract information from Foursquare response**

In [163]:
def getInfo(response, query):
    '''This function receives a request response from foursquare and the info you queried, 
    and returns a dataframe with the following information (columns):
    name, distance, address, latitude, longitude and geometry.
    :response: list, the response of the request from foursquare.
    :query: string, the name of your query in foursquare.
    '''
    
    geo_list = []
    
    for i in response["results"]:

        name = i["name"]
        address =  i["location"]["formatted_address"]
        lat = i["geocodes"]["main"]["latitude"]
        lon = i["geocodes"]["main"]["longitude"]
        distance = i["distance"]

        type_geo = {"typepoint": 
                              {"type": "Point", 
                               "coordinates": [lat, lon]}}

        geo_list.append({"query": query,"name":name, "address":address, "distance":distance, "lat":lat, "lon":lon, "type":type_geo})
    
    try:
        df = pd.DataFrame(geo_list)
        df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df["lon"], df["lat"]))
        df = df.drop(columns = 'type', axis = 1)
    except:
        return print("Empty response!")
    
    return df

**Function to get every information from Foursquare and return a concatenated df**

In [262]:
def getEverythingFoursquare(city):
    
    # Run all functions
    schools = getFoursquareCategory("school", city, 2000, 12055, limit=5)
    clubs = getFoursquare("night club", city, 1000, limit=5)
    starbucks = getFoursquare("starbucks", city, 1000, limit=5)
    basketball = getFoursquare("basketball stadium", city, 10000, limit=5)
    dog = getFoursquare("dog grooming", city, 5000, limit=5)    
    airport = getFoursquareCategory("international airport", city, 20000, 19040, limit=5)
    vegan = getFoursquareCategory("vegan restaurant", city, 1000, 13377, limit=5)
    
    # Transform to dataframes
    df_1 = getInfo(schools, "school")
    df_2 = getInfo(clubs, "club")
    df_3 = getInfo(starbucks, "starbucks")
    df_4 = getInfo(airport, "airport")
    df_5 = getInfo(vegan, "vegan restaurant")
    df_6 = getInfo(basketball, "basketball")
    df_7 = getInfo(dog, "dog hairdresser")
    
    # Concatenate dataframes
    df = pd.concat([df_1, df_2, df_3, df_4, df_5, df_6, df_7], ignore_index = True, axis = 0)
    
    return df

**Defining coordinates based on other companies' location**

In [68]:
sanfran = [37.781929, -122.404176]
newyork = [40.739930, -73.993049]
london = [51.517038, 0.139476]

## San Francisco

**Elementary schools**

In [233]:
# sanfran_schools = getFoursquare("elementary school",sanfran,1000,limit=5)

In [236]:
12057

12057

In [254]:
sanfran_schools_2 = getFoursquareCategory("school", sanfran, 10000, 12055, limit=5)

In [255]:
sanfran_schools_df_2 = getInfo(sanfran_schools, "elementary school")
sanfran_schools_df_2

,query,name,address,distance,lat,lon,geometry
0,elementary school,Bessie Carmichael Elementary School,"375 7th St (at Harrison St), San Francisco, CA...",657,37.776165,-122.406672,POINT (-122.40667 37.77616)


In [234]:
sanfran_schools_df = getInfo(sanfran_schools, "elementary school")
# sanfran_schools_df

In [235]:
sanfran_schools_df

,query,name,address,distance,lat,lon,geometry
0,elementary school,Bessie Carmichael Elementary School,"375 7th St (at Harrison St), San Francisco, CA...",657,37.776165,-122.406672,POINT (-122.40667 37.77616)


**Night clubs**

In [64]:
# sanfran_clubs = getFoursquare("night club",sanfran,1000,limit=5)

In [220]:
sanfran_clubs_df = getInfo(sanfran_clubs, "club")
# sanfran_clubs_df

In [221]:
sanfran_clubs_df

,query,name,address,distance,lat,lon,geometry
0,club,Covered Wagon Saloon,"917 Folsom St (at 5th), San Francisco, CA 94107",237,37.779890,-122.403666,POINT (-122.40367 37.77989)
1,club,The View Lounge,"780 Mission St, San Francisco, CA 94103",349,37.785072,-122.404258,POINT (-122.40426 37.78507)
2,club,The Factory,"370 4th St, San Francisco, CA 94107",351,37.780943,-122.400144,POINT (-122.40014 37.78094)
3,club,Monarch,"101 6th St (at Mission St), San Francisco, CA ...",398,37.780935,-122.408387,POINT (-122.40839 37.78093)
4,club,Nikitin & Reznikoff Obyebos,"788 Harrison St (4th), San Francisco, CA 94107",417,37.781219,-122.399515,POINT (-122.39951 37.78122)
5,club,Pura Club,"1015 Folsom St (Harriet), San Francisco, CA 94103",433,37.778128,-122.405784,POINT (-122.40578 37.77813)
6,club,1015 Folsom,"1015 Folsom St (at 6th St), San Francisco, CA ...",437,37.778128,-122.405784,POINT (-122.40578 37.77813)
7,club,Dance San Francisco,"1015 Folsom St, San Francisco, CA 94103",445,37.778128,-122.405784,POINT (-122.40578 37.77813)
8,club,Ten 15 Club,"1015 Folsom St, San Francisco, CA 94103",445,37.778128,-122.405784,POINT (-122.40578 37.77813)
9,club,Omg,"43 6th St (Between Market and Mission St.), Sa...",461,37.781727,-122.409401,POINT (-122.40940 37.78173)


**Starbucks**

In [102]:
# sanfran_starbucks = getFoursquare("starbucks",sanfran,1000,limit=5)

In [106]:
sanfran_starbucks_df = getInfo(sanfran_starbucks, "starbucks")
# sanfran_starbucks_df

In [218]:
sanfran_starbucks_df

,query,name,address,distance,lat,lon,geometry
0,starbucks,Starbucks,"120 4th St (at Mission St), San Francisco, CA ...",256,37.784199,-122.404135,POINT (-122.40413 37.78420)
1,starbucks,Starbucks,"120 4th St (at Mission St), San Francisco, CA ...",256,37.784199,-122.404135,POINT (-122.40413 37.78420)
2,starbucks,Starbucks,"789 Mission St (at 4th St), San Francisco, CA ...",297,37.784470,-122.404008,POINT (-122.40401 37.78447)
3,starbucks,Starbucks,"865 Market St (5th street), San Francisco, CA ...",388,37.784144,-122.407429,POINT (-122.40743 37.78414)
4,starbucks,Starbucks,"865 Market St (5th street), San Francisco, CA ...",388,37.784144,-122.407429,POINT (-122.40743 37.78414)
5,starbucks,Starbucks,"780 Market St (at 4th St), San Francisco, CA 9...",488,37.786145,-122.405612,POINT (-122.40561 37.78614)
6,starbucks,Starbucks,"780 Market St (at 4th St), San Francisco, CA 9...",488,37.786145,-122.405612,POINT (-122.40561 37.78614)
7,starbucks,Starbucks,"74 New Montgomery St (at Jessie St), San Franc...",715,37.787664,-122.401113,POINT (-122.40111 37.78766)
8,starbucks,Starbucks,"74 New Montgomery St (at Jessie St), San Franc...",715,37.787664,-122.401113,POINT (-122.40111 37.78766)
9,starbucks,Starbucks,"170 Ofarrell St, San Francisco, CA 94102",734,37.786534,-122.408216,POINT (-122.40822 37.78653)


**International Airport**

In [204]:
# sanfran_airport = getFoursquareCategory("international airport", sanfran, 20000, 19040, limit=5)

In [206]:
sanfran_airport_df = getInfo(sanfran_airport, "airport")
# sanfran_airport_df

In [217]:
sanfran_airport_df

,query,name,address,distance,lat,lon,geometry
0,airport,San Francisco International Airport (SFO),"at Link Rd, San Francisco, CA 94128",18415,37.624123,-122.395506,POINT (-122.39551 37.62412)
1,airport,Oakland International Airport (OAK),"1 Airport Dr, Oakland, CA 94621",18478,37.711700,-122.212997,POINT (-122.21300 37.71170)
2,airport,Norman Y. Mineta San Jose International Airpor...,"1701 Airport Blvd (at Skyport Dr), San Jose, C...",62298,37.367658,-121.927599,POINT (-121.92760 37.36766)


**Vegan restaurants**

In [124]:
# sanfran_vegan = getFoursquareCategory("vegan restaurant", sanfran, 1000, 13377, limit=5)

In [166]:
sanfran_vegan_df = getInfo(sanfran_vegan, "vegan restaurant")
# sanfran_vegan_df

In [207]:
sanfran_vegan_df

,query,name,address,distance,lat,lon,geometry
0,vegan restaurant,Mixt,"51 Yerba Buena Ln, San Francisco, CA 94103",424,37.785581,-122.404432,POINT (-122.40443 37.78558)
1,vegan restaurant,Ginger Bowls,"60 Morris St, San Francisco, CA 94107",566,37.776849,-122.402842,POINT (-122.40284 37.77685)
2,vegan restaurant,Plant B,"60 Morris St, San Francisco, CA 94107",582,37.776829,-122.402645,POINT (-122.40265 37.77683)
3,vegan restaurant,Thai Basil Cart,"60 Morris St (Bryant St), San Francisco, CA 94107",594,37.776768,-122.402622,POINT (-122.40262 37.77677)
4,vegan restaurant,Graze Salad Bar,"1028 Market St, San Francisco, CA 94102",614,37.781898,-122.411168,POINT (-122.41117 37.78190)
5,vegan restaurant,The Flying Falafel,"1051 Market St (at 6th St), San Francisco, CA ...",615,37.781240,-122.411217,POINT (-122.41122 37.78124)
6,vegan restaurant,Sweetgreen,"171 2nd St, San Francisco, CA 94105",766,37.787182,-122.398432,POINT (-122.39843 37.78718)
7,vegan restaurant,The Thonglor,"420 Geary St, San Francisco, CA 94102",802,37.787265,-122.410194,POINT (-122.41019 37.78726)
8,vegan restaurant,Millenium Restaurant,"580 Geary St, San Francisco, CA 94102",947,37.786880,-122.412957,POINT (-122.41296 37.78688)


**Basketball stadiums**

In [176]:
# sanfran_basketball = getFoursquare("basketball stadium", sanfran, 10000, limit=5)

In [177]:
sanfran_basketball_df = getInfo(sanfran_basketball, "basketball")
# sanfran_basketball_df

In [214]:
sanfran_basketball_df

,query,name,address,distance,lat,lon,geometry
0,basketball,Google Cloud Courtside Lounge 25,"San Francisco, CA 94158",2098,37.768190,-122.387852,POINT (-122.38785 37.76819)
1,basketball,Chase Center Stadium,"1 Warriors Way, San Francisco, CA 94158",2144,37.767935,-122.387335,POINT (-122.38733 37.76794)
2,basketball,United Club,"500 Terry A Francois Blvd, San Francisco, CA 9...",2158,37.767935,-122.387335,POINT (-122.38733 37.76794)
3,basketball,Kezar Pavilion,"755 Stanyan St, San Francisco, CA 94117",4652,37.767597,-122.453753,POINT (-122.45375 37.76760)


**Dobby the dog (hairdresser)**

In [210]:
# sanfran_dog = getFoursquare("dog grooming", sanfran, 10000, limit=5)

In [215]:
sanfran_dog_df = getInfo(sanfran_dog, "dog hairdresser")
# sanfran_dog_df

In [216]:
sanfran_dog_df

,query,name,address,distance,lat,lon,geometry
0,dog hairdresser,Bernie's Grooming,"1200 15th St (Harrison), San Francisco, CA 94103",1866,37.766871,-122.412862,POINT (-122.41286 37.76687)
1,dog hairdresser,Bow Wow Meow,"2150 Polk St (btw Vallejo & Broadway), San Fra...",2252,37.796544,-122.421711,POINT (-122.42171 37.79654)


In [263]:
# sanfran_df = getEverythingFoursquare(sanfran)

In [264]:
sanfran_df

,query,name,address,distance,lat,lon,geometry
0,school,C5 Children's School,"505 Van Ness Ave (McAllister), San Francisco, ...",1327,37.780887,-122.419280,POINT (-122.41928 37.78089)
1,school,Commodore Early Education School,"1 Trenton St, San Francisco, CA 94108",1507,37.794929,-122.409069,POINT (-122.40907 37.79493)
2,club,Covered Wagon Saloon,"917 Folsom St (at 5th), San Francisco, CA 94107",237,37.779890,-122.403666,POINT (-122.40367 37.77989)
3,club,The View Lounge,"780 Mission St, San Francisco, CA 94103",349,37.785072,-122.404258,POINT (-122.40426 37.78507)
4,club,The Factory,"370 4th St, San Francisco, CA 94107",351,37.780943,-122.400144,POINT (-122.40014 37.78094)
5,club,Monarch,"101 6th St (at Mission St), San Francisco, CA ...",398,37.780935,-122.408387,POINT (-122.40839 37.78093)
6,club,Nikitin & Reznikoff Obyebos,"788 Harrison St (4th), San Francisco, CA 94107",417,37.781219,-122.399515,POINT (-122.39951 37.78122)
7,starbucks,Starbucks,"120 4th St (at Mission St), San Francisco, CA ...",256,37.784199,-122.404135,POINT (-122.40413 37.78420)
8,starbucks,Starbucks,"120 4th St (at Mission St), San Francisco, CA ...",256,37.784199,-122.404135,POINT (-122.40413 37.78420)
9,starbucks,Starbucks,"789 Mission St (at 4th St), San Francisco, CA ...",297,37.784470,-122.404008,POINT (-122.40401 37.78447)


## South San Francisco

## New York

In [257]:
# newyork_df = getEverythingFoursquare(newyork)

In [258]:
newyork_df

,query,name,address,distance,lat,lon,geometry
0,school,Buckle My Shoe,"230 W 13th St (7-8thave), New York, NY 10011",750,40.738264,-74.001865,POINT (-74.00186 40.73826)
1,school,The Jack And Jill School,"209 E 16th St, New York, NY 10003",886,40.734558,-73.985344,POINT (-73.98534 40.73456)
2,school,Aleph Bet NY,"240 E 27th St, New York, NY 10016",1046,40.743134,-73.981444,POINT (-73.98144 40.74313)
3,school,Stork and Cradle,"171 Madison Ave (33rd), New York, NY 10016",1161,40.747297,-73.983092,POINT (-73.98309 40.74730)
4,school,Starchild Ny,"435 E 6th St, New York, NY 10009",1723,40.725843,-73.984479,POINT (-73.98448 40.72584)
5,club,Jimmy Antons Salsa Social Sundays,"39-47 E 19 St (5th And 6th Avenue), New York, ...",38,40.740219,-73.993470,POINT (-73.99347 40.74022)
6,club,The Imperial,"17 W 19th St, New York, NY 10011",61,40.739603,-73.992310,POINT (-73.99231 40.73960)
7,club,The VIP Club for Adult Entertainment NYC,"20 W 20th St (btw 5th & 6th Ave), New York, NY...",64,40.740190,-73.992552,POINT (-73.99255 40.74019)
8,club,The Limelight,"660 Avenue of the Americas, New York, NY 10010",147,40.741127,-73.993811,POINT (-73.99381 40.74113)
9,club,Hunkamania Male Strippers NYC,"45 W 21st St, New York, NY 10010",168,40.741426,-73.992731,POINT (-73.99273 40.74143)


**Basketball stadium**

In [174]:
# newyork_basketball = getFoursquare("basketball stadium", newyork, 10000, limit=10)

In [175]:
newyork_basketball_df = getInfo(newyork_basketball, "basketball")
# newyork_basketball_df

In [183]:
newyork_dog = getFoursquare("dog grooming", newyork, 10000, limit=1)

In [184]:
newyork_dog

{'results': [{'fsq_id': '4dbc69011e72b351cab27036',
   'categories': [{'id': 11134,
     'name': 'Pet Grooming Service',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
      'suffix': '.png'}},
    {'id': 17110,
     'name': 'Pet Supplies Store',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/pet_store_',
      'suffix': '.png'}}],
   'chains': [],
   'distance': 3065,
   'geocodes': {'main': {'latitude': 40.75687, 'longitude': -73.964245},
    'roof': {'latitude': 40.75687, 'longitude': -73.964245}},
   'link': '/v3/places/4dbc69011e72b351cab27036',
   'location': {'address': '1001 1st Ave',
    'census_block': '360610108013000',
    'country': 'US',
    'cross_street': '',
    'dma': 'New York',
    'formatted_address': '1001 1st Ave, New York, NY 10022',
    'locality': 'New York',
    'neighborhood': ['Midtown'],
    'postcode': '10022',
    'region': 'NY'},
   'name': "Reme's Oggi Pets",
   'related_places': {},
   'timezo

## London

In [228]:
london_df = getEverythingFoursquare(london)

Empty response!
Empty response!
Empty response!
Empty response!
Empty response!
Empty response!
Empty response!


ValueError: All objects passed were None